# OpenAI APIs - Vision

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/guides/vision).
This tutorial covers the vision APIs for vision language models.

SGLang supports various vision language models such as Llama 3.2, LLaVA-OneVision, Qwen2.5-VL, Gemma3 and [more](https://docs.sglang.ai/references/supported_models): 
- [meta-llama/Llama-3.2-11B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct)  
- [lmms-lab/llava-onevision-qwen2-72b-ov-chat](https://huggingface.co/lmms-lab/llava-onevision-qwen2-72b-ov-chat)  
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct)
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it)
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V)
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2)

As an alternative to the OpenAI API, you can also use the [SGLang offline engine](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py).

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

**Remember to add** `--chat-template llama_3_vision` **to specify the [vision chat template](https://docs.sglang.ai/backend/openai_api_vision.html#Chat-Template), otherwise, the server will only support text (images won’t be passed in), which can lead to degraded performance.**

We need to specify `--chat-template` for vision language models because the chat template provided in Hugging Face tokenizer only supports text.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

vision_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-11B-Vision-Instruct \
    --chat-template=llama_3_vision
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-31 07:11:19] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-11B-Vision-Instruct', chat_template='llama_3_vision', completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=39606, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=469085838, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', 

[2025-03-31 07:11:24] Use chat template for the OpenAI-compatible API server: llama_3_vision


[2025-03-31 07:11:30 TP0] Overlap scheduler is disabled for multimodal models.
[2025-03-31 07:11:30 TP0] Automatically reduce --mem-fraction-static to 0.836 because this is a multimodal model.
[2025-03-31 07:11:30 TP0] Automatically turn off --chunked-prefill-size for mllama.
[2025-03-31 07:11:30 TP0] Init torch distributed begin.


[2025-03-31 07:11:30 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-31 07:11:30 TP0] Load weight begin. avail mem=78.58 GB


[2025-03-31 07:11:33 TP0] Using model weights format ['*.safetensors']


...

[2025-03-31 07:11:40 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-03-31 07:11:40] INFO:     Started server process [591914]
[2025-03-31 07:11:40] INFO:     Waiting for application startup.
[2025-03-31 07:11:40] INFO:     Application startup complete.
[2025-03-31 07:11:40] INFO:     Uvicorn running on http://127.0.0.1:39606 (Press CTRL+C to quit)


[2025-03-31 07:11:41] INFO:     127.0.0.1:51934 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-31 07:11:41] INFO:     127.0.0.1:51946 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-31 07:11:41 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-31 07:11:43] INFO:     127.0.0.1:51948 - "POST /generate HTTP/1.1" 200 OK
[2025-03-31 07:11:43] The server is fired up and ready to roll!


## Using cURL

Once the server is up, you can send test requests using curl or requests.

In [2]:
import subprocess

curl_command = f"""
curl -s http://localhost:{port}/v1/chat/completions \\
  -d '{{
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
      {{
        "role": "user",
        "content": [
          {{
            "type": "text",
            "text": "What’s in this image?"
          }},
          {{
            "type": "image_url",
            "image_url": {{
              "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
            }}
          }}
        ]
      }}
    ],
    "max_tokens": 300
  }}'
"""

response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)


response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)

[2025-03-31 07:11:49 TP0] Prefill batch. #new-seq: 1, #new-token: 6463, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-31 07:11:50 TP0] Decode batch. #running-req: 1, #token: 6496, token usage: 0.32, gen throughput (token/s): 3.90, #queue-req: 0, 


[2025-03-31 07:11:51 TP0] Decode batch. #running-req: 1, #token: 6536, token usage: 0.32, gen throughput (token/s): 56.35, #queue-req: 0, 


[2025-03-31 07:11:51 TP0] Decode batch. #running-req: 1, #token: 6576, token usage: 0.32, gen throughput (token/s): 52.00, #queue-req: 0, 


[2025-03-31 07:11:52 TP0] Decode batch. #running-req: 1, #token: 6616, token usage: 0.32, gen throughput (token/s): 62.84, #queue-req: 0, 
[2025-03-31 07:11:52] INFO:     127.0.0.1:51954 - "POST /v1/chat/completions HTTP/1.1" 200 OK


...

[2025-03-31 07:11:55 TP0] Decode batch. #running-req: 1, #token: 6622, token usage: 0.32, gen throughput (token/s): 58.54, #queue-req: 0, 


[2025-03-31 07:11:56 TP0] Decode batch. #running-req: 1, #token: 6662, token usage: 0.33, gen throughput (token/s): 63.55, #queue-req: 0, 


[2025-03-31 07:11:57 TP0] Decode batch. #running-req: 1, #token: 6702, token usage: 0.33, gen throughput (token/s): 49.65, #queue-req: 0, 


[2025-03-31 07:11:58 TP0] Decode batch. #running-req: 1, #token: 6742, token usage: 0.33, gen throughput (token/s): 36.09, #queue-req: 0, 


[2025-03-31 07:11:58] INFO:     127.0.0.1:39238 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using Python Requests

In [3]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    "max_tokens": 300,
}

response = requests.post(url, json=data)
print_highlight(response.text)

[2025-03-31 07:11:59 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 6462, token usage: 0.32, #running-req: 0, #queue-req: 0, 


[2025-03-31 07:12:00 TP0] Decode batch. #running-req: 1, #token: 6483, token usage: 0.32, gen throughput (token/s): 20.56, #queue-req: 0, 
[2025-03-31 07:12:00] INFO:     127.0.0.1:39246 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using OpenAI Python Client

In [4]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print_highlight(response.choices[0].message.content)

[2025-03-31 07:12:00 TP0] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 6452, token usage: 0.32, #running-req: 0, #queue-req: 0, 


[2025-03-31 07:12:01] INFO:     127.0.0.1:39248 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Multiple-Image Inputs

The server also supports multiple images and interleaved text and images if the model supports it.

In [5]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/sgl-project/sglang/main/assets/logo.png",
                    },
                },
                {
                    "type": "text",
                    "text": "I have two very different images. They are not related at all. "
                    "Please describe the first image in one sentence, and then describe the second image in another sentence.",
                },
            ],
        }
    ],
    temperature=0,
)

print_highlight(response.choices[0].message.content)

[2025-03-31 07:12:03 TP0] Prefill batch. #new-seq: 1, #new-token: 12895, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-31 07:12:03 TP0] Decode batch. #running-req: 1, #token: 12905, token usage: 0.63, gen throughput (token/s): 10.76, #queue-req: 0, 


[2025-03-31 07:12:04 TP0] Decode batch. #running-req: 1, #token: 12945, token usage: 0.63, gen throughput (token/s): 63.60, #queue-req: 0, 


[2025-03-31 07:12:04] INFO:     127.0.0.1:42126 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(vision_process)

[2025-03-31 07:12:04] Child process unexpectedly failed with an exit code 9. pid=592394
[2025-03-31 07:12:04] Child process unexpectedly failed with an exit code 9. pid=592321


[2025-03-31 07:12:05] Child process unexpectedly failed with an exit code 9. pid=594712
[2025-03-31 07:12:05] Child process unexpectedly failed with an exit code 9. pid=594702
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...


[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] Child process unexpectedly failed with an exit code 9. pid=594719
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-31 07:12:05] SIGTERM received. signum=None frame=None. Draining requests and sh

## Chat Template

As mentioned before, if you do not specify a vision model's `--chat-template`, the server uses Hugging Face's default template, which only supports text.

We list popular vision models with their chat templates:

- [meta-llama/Llama-3.2-Vision](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct) uses `llama_3_vision`.
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct) uses `qwen2-vl`.
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it) uses `gemma-it`.
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V) uses `minicpmv`.
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2) uses `deepseek-vl2`.
- [LlaVA-OneVision](https://huggingface.co/lmms-lab/llava-onevision-qwen2-7b-ov) uses `chatml-llava`.
- [LLaVA-NeXT](https://huggingface.co/collections/lmms-lab/llava-next-6623288e2d61edba3ddbf5ff) uses `chatml-llava`.
- [Llama3-LLaVA-NeXT](https://huggingface.co/lmms-lab/llama3-llava-next-8b) uses `llava_llama_3`.
- [LLaVA-v1.5 / 1.6](https://huggingface.co/liuhaotian/llava-v1.6-34b) uses `vicuna_v1.1`.